In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from flask import Flask, request
from flask_restful import Resource, Api
from sqlalchemy import create_engine
from json import dumps
from flask_jsonpify import jsonify

In [ ]:
flights = pd.read_csv('flights.csv')
#flights1 = flights.iloc[:1000]                    # Tried with a smaller database to get results faster initially
airports = pd.read_csv('airports.csv')
airlines = pd.read_csv('airlines.csv')

In [ ]:
#Find the correlation between route distance and flight delays

class Index(Resource):
    def get(self):
        answer = "The endpoints are: \n"
        answer += "correlation,origin_delayed_airports,destination_delayed_airports,avoid_airlines,corr_matrix \n"
        answer += "1. correlation: Displays the correlation between route distance and flight delays \n"
        answer += "2. origin_delayed_airports: Displays the airports that are part of the most delayed flights (source) \n"
        answer += "3. destination_delayed_airports: Displays the airports that are part of the most delayed flights (destination) \n"
        answer += "4. avoid_airlines: Displays the airlines you should fly on to avoid significant delays \n"
        answer += "5. corr_matrix: Are there any interesting observations you can make from the dataset? \n"
        return (answer)

In [ ]:
#Find the correlation between route distance and flight delays

class Corr(Resource):
    def get(self):
        X = flights['ARRIVAL_DELAY']
        Y = flights['DISTANCE']
        corr = X.corr(Y)
        return ("The value of the correlation between route distance and flight delays is " + str(corr))

In [ ]:
#Which airports are part of the most delayed flights (either source or destination)?

class Origin_delayed_airports(Resource):
    def get(self):
        conn = db_connect.connect() # connect to database
        query = conn.execute("select Origin_Airport, departure_delays from (select a.airport as Origin_Airport, count(f.origin_airport) as departure_delays from flights f inner join airports a on f.origin_airport=a.iata_code where f.departure_delay > 0 group by f.origin_airport order by departure_delays desc)t1 limit 10") # This line performs query and returns json result
        return {'Delayed airports': [i[0] for i in query.cursor.fetchall()]} # Fetches first column that is Employee ID

In [ ]:
#Which airports are part of the most delayed flights (either source or destination)?

class Destination_delayed_airports(Resource):
    def get(self):
        conn = db_connect.connect() # connect to database
        query = conn.execute("select Destination_Airport, arrival_delays from (select a.airport as Destination_Airport, count(f.destination_airport) as arrival_delays from flights f inner join airports a on f.destination_airport=a.iata_code where f.arrival_delay > 0 group by f.destination_airport order by arrival_delays desc)t2 limit 10") # This line performs query and returns json result
        return {'Delayed airports': [i[0] for i in query.cursor.fetchall()]} # Fetches first column that is Employee ID

In [ ]:
#Which airline should you fly on to avoid significant delays?

class Avoid_airlines(Resource):
    def get(self):
        conn = db_connect.connect() # connect to database
        query = conn.execute("select airline, count(airline) from flights where departure_delay > 0 or arrival_delay > 0 group by airline order by count(airline)") # This line performs query and returns json result
        return {'The airlines to fly on to avoid significant delays are': [i[0] for i in query.cursor.fetchall()]} # Fetches first column that is Employee ID

In [ ]:
#Are there any interesting observations you can make from the dataset?

class Interesting(Resource):
    def get(self):
        return ([(airline + " " + str((flights1[flights1['AIRLINE'] == airline])['ARRIVAL_DELAY'].mean())) for airline in flights1['AIRLINE'].unique()])
    

In [ ]:
engine = create_engine('sqlite:///Stackline.db', echo=False)
airlines.to_sql('airlines', con=engine)
airports.to_sql('airports', con=engine)
flights.to_sql('flights', con=engine)
#flights1.to_sql('flights', con=engine)

db_connect = create_engine('sqlite:///Stackline.db')
app = Flask(__name__)
api = Api(app)

api.add_resource(Index, '/') 
api.add_resource(Corr, '/correlation')
api.add_resource(Origin_delayed_airports, '/origin_delayed_airports')
api.add_resource(Destination_delayed_airports, '/destination_delayed_airports')
api.add_resource(Avoid_airlines, '/avoid_airlines')
api.add_resource(Interesting,'/interesting')

if __name__ == '__main__':
     app.run(port=5002)